In [1]:
import sys, os
sys.path.insert(0, os.path.abspath("../.."))
%load_ext ElasticNotebook

In [2]:
%%RecordEvent
import time
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

In [3]:
%%RecordEvent
train_data = pd.read_csv("./data/kaggle/titanic/input/train.csv")
print(len(train_data))
train_data.head()

891


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
%%RecordEvent
test_data = pd.read_csv("./data/kaggle/titanic/input/test.csv")
print(len(test_data))
test_data.head()

418


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [5]:
%%RecordEvent
women = train_data.loc[train_data.Sex == 'female']["Survived"]
rate_women = sum(women)/len(women)

print("% of women who survived:", rate_women)

% of women who survived: 0.7420382165605095


In [6]:
%%RecordEvent
men = train_data.loc[train_data.Sex == 'male']["Survived"]
rate_men = sum(men)/len(men)

print("% of men who survived:", rate_men)

% of men who survived: 0.18890814558058924


In [7]:
%%RecordEvent
# 前処理
# データの前処理
features = ["Pclass", "Name", "Sex", "Age", "SibSp", "Parch", "Ticket", "Fare", "Cabin", "Embarked"]
X = pd.get_dummies(train_data[features])
y = train_data["Survived"]

# 列を揃える
X_test = pd.get_dummies(test_data[features])
X_test = X_test.reindex(columns=X.columns, fill_value=0)

# 訓練データと検証データの分割
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=1)

In [8]:
%%RecordEvent
# start_time = time.time()

# モデルの学習
model = RandomForestClassifier(
    n_estimators=10000,       # 決定木の数を増加
    max_depth=None,          # 木の深さを無制限
    min_samples_split=2,     # 分割する最小サンプル数
    min_samples_leaf=1,      # 葉ノードに必要な最小サンプル数
    random_state=1,
    n_jobs=-1                # 並列処理で計算時間を短縮
)
model.fit(X_train, y_train)

# print(f"--- {time.time() - start_time} seconds ---")

RandomForestClassifier(n_estimators=10000, n_jobs=-1, random_state=1)

In [9]:
%%RecordEvent
# 検証
# 訓練データ上の性能
train_predictions = model.predict(X_train)
print("Classification Report (Training Data):")
print(classification_report(y_train, train_predictions))

# 検証データ上の性能
val_predictions = model.predict(X_val)
print("Classification Report (Validation Data):")
print(classification_report(y_val, val_predictions))

Classification Report (Training Data):
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       443
           1       1.00      1.00      1.00       269

    accuracy                           1.00       712
   macro avg       1.00      1.00      1.00       712
weighted avg       1.00      1.00      1.00       712

Classification Report (Validation Data):
              precision    recall  f1-score   support

           0       0.76      0.96      0.85       106
           1       0.91      0.56      0.69        73

    accuracy                           0.80       179
   macro avg       0.84      0.76      0.77       179
weighted avg       0.82      0.80      0.79       179



In [10]:
%%RecordEvent
# テストデータへの予測
test_predictions = model.predict(X_test)
output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': test_predictions})
output.to_csv('./data/kaggle/titanic/output/submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!


In [11]:
%Checkpoint checkpoints/large_scale_program.pickle

migration speed (bps) (total_bytes / (total_read_time + total_write_time * alpha)): 931361978.2992682
|- total bytes: 799920000
|- total read time: 9.298324584960938e-05
|- total write time: 0.8587782382965088
|- alpha: 1
---------------------------
all variables:
name: RandomForestClassifier
name: classification_report
name: train_test_split
name: pd
name: time
name: train_data
name: test_data
name: women
name: rate_women
name: men
name: rate_men
name: X_train
name: X_test
name: y_val
name: X
name: y
name: X_val
name: y_train
name: features
name: model
name: train_predictions
name: val_predictions
name: output
name: test_predictions
---------------------------
variables to migrate:
name: output, size: 6852
name: time, size: 72
name: rate_men, size: 24
name: model, size: 48
name: women, size: 5056
name: X_train, size: 1261696
name: RandomForestClassifier, size: 1704
name: X_test, size: 5316706
name: y_val, size: 2896
name: X, size: 1571888
name: rate_women, size: 24
name: y, size: 7292